# xenium tutorial

In [1]:
import scispy as scis
import pandas as pd
import seaborn as sns
import scanpy as sc
import anndata as ad
from anndata import AnnData
import spatialdata as sd
from spatialdata import SpatialData
from spatialdata import polygon_query
from matplotlib import pyplot as plt

### load raw data and perform scanpy analysis

In [ ]:
sdata = scis.io.load_xenium('xenium_output_dir')
scis.pp.run_scanpy(sdata)

### annotate your spatial cells using a single-cell reference

In [2]:
scref = sc.read_h5ad('scref.h5ad')
scis.pp.scvi_annotate(sdata['table'], 
                      scref, 
                      label_ref='celltype', 
                      label_key='celltype')

### add anatomical regions

In [ ]:
scis.tl.add_to_shapes(sdata, 
                      shape_key='shapes', 
                      scale_factor=1/0.2125, 
                      target_coordinates='global', 
                      shape_file='coordinates.csv')

sdata['shapes'].name = sdata['shapes'].name.astype('category')

f, ax = plt.subplots(figsize=(8, 5))
sdata.pl.render_images(cmap='Blues').pl.show(ax=ax)
sdata.pl.render_shapes(elements='shapes', color='name', fill_alpha=0.8).pl.show("global",ax=ax)
ax.set_title('my_sample')
plt.show()

In [ ]:
sdata['table'].obs['anatomy'] = 'unknown'
for i, shape in sdata['shapes'].iterrows():
    try:
        sd_shape = polygon_query(sdata, polygon=shape.geometry, target_coordinate_system="global")
        sdata['table'].obs.anatomy[sdata['table'].obs['cell_id'].isin(sd_shape.table.obs['cell_id'])] = shape['name']
    except:
        print(shape['name'], " contains invalid coordinates")

### save sdata object

In [ ]:
sdata.write("sdata_1")

#import spatialdata_xenium_explorer
#spatialdata_xenium_explorer.write("explorer_1", sdata, shapes_key='cell_boundaries', points_key='transcripts', gene_column='feature_name', layer='counts', polygon_max_vertices=40, mode='-it')

### concatenate sdata objects

In [ ]:
adatas = []
samples = ['sdata_1','sdata_2','sdata_3','sdata_4']
for s in samples:
    sdata = SpatialData.read(s)
    adatas += [sdata.table]

adata = ad.concat(adatas)

### visualize cell type proportions

In [ ]:
scis.tl.scis_prop(adata, 
                  celltype='celltype', 
                  zone='anatomy', 
                  top=10, 
                  replicate='sample', 
                  condition='genotype', 
                  condition_order=['KO','CTRL'],
                  figsize=(6,4))

### run pseudobulk analysis

In [ ]:
df = scis.tl.run_pseudobulk(sdata['table'], 
                            replicate_key='sample', 
                            cond_key='genotype', 
                            cond_1='KO', 
                            cond_2='CTRL', 
                            groups_key='celltype')

# plot results as seaborn heatmap
pivlfc = pd.pivot_table(df, values=["log2FoldChange"], index=["index"], columns=['celltype'], fill_value=0)
sns.clustermap(pivlfc, cmap="vlag", figsize=(2, 8))